# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# Q1:  What test is appropriate for this problem? Does the CLT apply?

For this problem, we want to see if each group, the 'b' and 'w' sounding resumes, yeild similar returns toward a call. Since we have two groups and want to see if each are treated similarly, I am going to use a permutation test. The CLT does apply in this situation since we are assuming that each entry of 'b' or 'w' is independently assigned (from the identical resumes) and we would expect a normal distribution of results from this data.

# Q2: What are the null and alternate hypotheses?

The null hypothesis is that the two data sets have identical rates or proportions of call returns. That being said, their difference should equate to 0 in our null hypothesis. The alternative hypothesis is that the difference of these values do not equal 0. These hypotheses are described below: 
<br>
<br>
<center>$H_{0}$: $μ_{w}$ - $μ_{b}$ = 0</center>
<center>$H_{a}$: $μ_{w}$ - $μ_{b}$ ≠ 0</center>

# Q3: Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [5]:
#Define parts of list that are white- or black-sounding: w, b
w = data[data.race=='w']
b = data[data.race=='b']

In [6]:
#Determine sample proportion of returned calls with white-sounding names: w_rate
w_returns = w.call.sum()
w_attempts = w.call.count()
w_rate = w_returns/w_attempts

print(w_rate)

0.09650924024640657


In [7]:
#Determine sample proportion of returned calls with black-sounding names: b_rate
b_returns = b.call.sum()
b_attempts = b.call.count()
b_rate = b_returns/b_attempts

print(b_rate)

0.06447638603696099


## Part 1: Bootstrap Approach

We will use a bootstrap method to collect data and perform a hypothesis test. This essentially takes small sample sets (with replacement) from within the sample data, and calculates the mean difference of returns between white- and black-sounding names. By plotting these results, we can determine the p% confidence interval of our observed test statistic.

In [8]:
#Reconstruct column 'race' to be boolean, where w = 1, b = 0
def boolean_race(row):
    if row['race'] == 'w':
        return 1
    return 0

In [9]:
#Apply function to race column, save as bool_race dataframe
res = data.apply(boolean_race, axis=1)
bool_race = res.to_frame('bool_race')

In [10]:
#Add bool_race column to dataframe and select necessary columns: call_race_data
new_data = pd.concat([data, bool_race], axis=1)
call_race_data = new_data[['call', 'bool_race']]
call_race_data.head()

,call,bool_race
0,0.0,1
1,0.0,1
2,0.0,0
3,0.0,0
4,0.0,1


In [11]:
#Define data of interest and turn to arrays: call, race
call = call_race_data['call'].values
race = call_race_data['bool_race'].values

Since these columns, 'call' and 'bool_race', are dependent on each other, we have to use a pair-wise bootstrap method. This will resample the data in pairs and compute the slope and intercept from the sampled data. From there, we can compute p-values and confidence intervals.

In [12]:
#Define how we draw bootstrap pairs
def draw_bs_pairs_linreg(x, y, size=1):
    """Perform pairs bootstrap for linear regression."""
    #Set up array of indices to sample from: inds
    inds = np.arange(len(x))
    
    #Initialize replicates: bs_slope_reps, bs_intercept_reps
    bs_slope_reps = np.empty(size)
    bs_intercept_reps = np.empty(size)
    
    #Generate replicates
    for i in range(size):
        bs_inds = np.random.choice(inds, size=len(inds))
        bs_x, bs_y = x[bs_inds], y[bs_inds]
        bs_slope_reps[i], bs_intercept_reps[i] = np.polyfit(bs_x, bs_y, 1)

    return bs_slope_reps, bs_intercept_reps

In [13]:
#Draw 10000 pairs
draw_bs_pairs_linreg(call, race, size=10000)

(array([0.07690997, 0.10167087, 0.11131755, ..., 0.14439998, 0.0962982 ,
        0.09741227]),
 array([0.47321228, 0.49138124, 0.49131403, ..., 0.48992838, 0.49092539,
        0.50067385]))

In [16]:
#Plot the first 200 bootstrap pairs and visualize the linear relationships

# Generate array of x-values for bootstrap lines: x
x = np.array([0, 100])

# Plot the bootstrap lines
for i in range(100):
    _ = plt.plot(x, bs_slope_reps[i]*x + bs_intercept_reps[i],
                 linewidth=0.5, alpha=0.1, color='red')

# Plot the data
_ = plt.plot(call, race, marker='.', linestyle='none')

# Label axes, set the margins, and show the plot
_ = plt.xlabel('call')
_ = plt.ylabel('race where w=1')
plt.margins(0.02)
plt.show()

NameError: name 'bs_slope_reps' is not defined

## Part 2: Frequentist Approach

We will perform a permutation hypothesis test. To do this, we will tkae 1,000 permutation samples. Each sample will include the following procedure. 
<br>
<br>
Take the original data and permute (shuffle) the entries in the 'call' column. Then, assign the first half to be "white-sounding" and the second to be "black-sounding." We will then split the data into permuted samples: perm_w and perm_b. Plot each difference of means of those two values 1,000 times in a histogram. Finally, we will see how extreme our observed sample data value is in relation to this distribution of permutation tests.

In [ ]:
#Write a script that calculates permuted data mean differences

#Initialize empty array of replicates
perm_reps = np.empty(1000)

#Establish random seed for reproducability
np.random.seed(42)

#Draw replicates
for i in range(1000):
    #Permute call_data
    if i <= 1:
        permuted_data = np.random.permutation(call_data)
        perm_reps[i] = np.mean(permuted_data[:len(w)]) - np.mean(permuted_data[len(w):])
    #Take permutations of most recentle permuted data
    else:
        permuted_data = np.random.permutation(permuted_data)
        perm_rep[i] = np.mean(permuted_data[:len(w)]) - np.mean(permuted_data[len(w):])

In [ ]:
#Plot histogram of array
plt.hist(perm_reps, bins=50, density=True)

In [ ]:
#Calculate observed sample mean difference
obs_samp_dif = w_rate - b_rate
print(obs_samp_dif)

In [ ]:
#Calculate how many standard deviations our observed value is from the mean
perm_std = np.std(perm_reps)
stds = obs_samp_dif/perm_std

print(stds, ' standard deviations from the mean')

In [ ]:
#Calculate p-value of obs_samp_dif
p = (obs_samp_dif - np.mean(perm_reps))/len(perm_reps)
print('p-value = ', p)

In conclusion, our observed difference of the mean rates of returned applications between white-sounding and black-sounding resumes was 0.03203. As we drew permutation samples and plotted the histogram of mean differences, we see that this observed value is far from the mean; in fact, this is over 4 standard deviations below what we would expect. This gives us a p-value of 3.207e-05, which is extremely low, guiding us to assess that our observed data does not actually follow a non-biased rate or return of resumes, as we assumed in our original hypothesis.